In [99]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [100]:
test_csv = pd.read_csv('test_data (1).csv') 
train_csv = pd.read_csv('train_data (1).csv')

In [101]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [102]:
#penghapusan stopword dan lemmatisasi
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [103]:
train_csv.tail()

,0,1
24995,With this movie being the only Dirty Harry mov...,1
24996,Any screen adaptation of a John Grisham story ...,1
24997,This film captured my heart from the very begi...,1
24998,A deplorable social condition triggers off the...,1
24999,Always enjoy the great acting of Drew Barrymor...,1


In [104]:
train_X_non = train_csv['0']    # '0' mengacu pada teks ulasan
train_y = train_csv['1']   # '1' sesuai dengan Label (1 - positif dan 0 - negatif)
test_X_non = test_csv['0']
test_y = test_csv['1']
train_X=[]
test_X=[]

# Teks Prepocessing


In [105]:
for i in range(0, len(train_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', train_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

In [106]:
for i in range(0, len(test_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', test_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [107]:
train_X[10]

'eddie murphy delirious far funniest thing ever see life compare movie garuntee decide delirious funniest movie ever movie hr min throughout time barely moment laughing laugh hour replaying punch line head eddie murphy given many funny performance career hr trading place beverly hill cop raw coming america nutty professor shrek etc far hilarious moment seen movie many time funnier every time never loses edge day forward every great stand performance emulated delirious two thumb'

Kita dapat melihat bahwa tanda baca dihilangkan dan semua stopword juga dihilangkan. Teks ini sekarang dapat digunakan untuk melatih classifier. 

#  TF-IDF Vectorizer

In [108]:
tf_idf = TfidfVectorizer()
#menerapkan tf idf ke data pelatihan
X_train_tf = tf_idf.fit_transform(train_X)
#menerapkan tf idf ke data pelatihan

X_train_tf = tf_idf.transform(train_X)

In [109]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape) #memeriksa sample dan fitur

n_samples: 25000, n_features: 65498


In [110]:
#mengubah data uji menjadi matriks tf-idf
X_test_tf = tf_idf.transform(test_X)

In [111]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 25000, n_features: 65498


# Naive Bayes Classifier

In [112]:
#pengklasifikasi naive bayes
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

MultinomialNB()

In [113]:
#predicted y
y_pred = naive_bayes_classifier.predict(X_test_tf)

Prediksi selesai. Sekarang,mencetak laporan klasifikasi. 

In [114]:
print(metrics.classification_report(test_y, y_pred, target_names=['Positive', 'Negative']))

              precision    recall  f1-score   support

    Positive       0.80      0.88      0.84     12500
    Negative       0.87      0.78      0.82     12500

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
weighted avg       0.83      0.83      0.83     25000



In [115]:
print("Confusion matrix:")   #periksa matriks confusion
print(metrics.confusion_matrix(test_y, y_pred))

Confusion matrix:
[[10990  1510]
 [ 2693  9807]]


# Uji prediksi pada reviews clasifier TF IDF

In [116]:
#Melakukan Tes Predikai
test = [" really looking forward movie based preview press received viewing terribly disappointed slum totally unfunny film mixed todd solondz type disturbing family reality sans todd brand humor story drag along scene worse last maybe missed point film example every girl growing experience glad man "]

In [117]:
review = re.sub('[^a-zA-Z]', ' ', test[0])
review = review.lower()
review = review.split()
review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
test_processed =[ ' '.join(review)]

In [118]:
test_processed  #melihat teks yang diproses

['really looking forward movie based preview press received viewing terribly disappointed slum totally unfunny film mixed todd solondz type disturbing family reality sans todd brand humor story drag along scene worse last maybe missed point film example every girl growing experience glad man']

In [119]:
test_input = tf_idf.transform(test_processed)
test_input.shape

(1, 65498)

In [120]:
#0= bad review
#1= good review
res=naive_bayes_classifier.predict(test_input)[0]
if res==1:
    print("Good Review")
elif res==0:
    print("Bad Review")

Bad Review
